# Concatenating COSA Datasets LOL

## Imports

In [1]:
import pandas as pd

In [2]:
#initial dataset 
air1 = pd.read_csv('med_center_air1.csv')

In [3]:
#additional dataset
air2 = pd.read_csv('med_center_air.csv')

In [4]:
#check out datatypes and nulls
air1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23379 entries, 0 to 23378
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DateTime        23379 non-null  object 
 1   Sensor_id       23379 non-null  object 
 2   Vendor          23379 non-null  object 
 3   SensorModel     23379 non-null  object 
 4   LAT             23379 non-null  float64
 5   LONG            23379 non-null  float64
 6   Zone            23379 non-null  object 
 7   Pm1_0           23379 non-null  int64  
 8   Pm2_5           23379 non-null  int64  
 9   Pm10            23379 non-null  int64  
 10  SO2             23379 non-null  int64  
 11  O3              23379 non-null  int64  
 12  CO              23379 non-null  int64  
 13  NO2             23379 non-null  int64  
 14  AlertTriggered  17619 non-null  object 
 15  SensorStatus    23379 non-null  object 
dtypes: float64(2), int64(7), object(7)
memory usage: 2.9+ MB


In [5]:
#checkout datatypes and nulls
air2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106287 entries, 0 to 106286
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DateTime        106287 non-null  object 
 1   Sensor_id       106287 non-null  object 
 2   Vendor          106287 non-null  object 
 3   SensorModel     106287 non-null  object 
 4   LAT             106287 non-null  float64
 5   LONG            106287 non-null  float64
 6   Zone            106287 non-null  object 
 7   Pm1_0           106287 non-null  int64  
 8   Pm2_5           106287 non-null  int64  
 9   Pm10            106287 non-null  int64  
 10  SO2             106287 non-null  int64  
 11  O3              106287 non-null  float64
 12  CO              106287 non-null  int64  
 13  NO2             106287 non-null  int64  
 14  AlertTriggered  86703 non-null   object 
 15  SensorStatus    106287 non-null  object 
dtypes: float64(3), int64(6), object(7)
memory usage: 13.0+ M

- O3 is a float in air 2, need to make them both floats

In [6]:
air1.astype({'O3': 'float64'}).dtypes

DateTime           object
Sensor_id          object
Vendor             object
SensorModel        object
LAT               float64
LONG              float64
Zone               object
Pm1_0               int64
Pm2_5               int64
Pm10                int64
SO2                 int64
O3                float64
CO                  int64
NO2                 int64
AlertTriggered     object
SensorStatus       object
dtype: object

In [7]:
air1.astype({'O3': 'float64'})

,DateTime,Sensor_id,Vendor,SensorModel,LAT,LONG,Zone,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,AlertTriggered,SensorStatus
0,2021-05-16T00:01:26,63e0b0eae3495a39,ATT,AQM-01-00-01-00-01-00-GY,29.51007,-98.580288,Medical Center,8,7,8,0,0.0,0,0,None,Normal
1,2021-05-16T00:04:26,63e0b0eae3495a39,ATT,AQM-01-00-01-00-01-00-GY,29.51007,-98.580288,Medical Center,8,7,8,0,0.0,0,0,None,Normal
2,2021-05-16T00:07:26,63e0b0eae3495a39,ATT,AQM-01-00-01-00-01-00-GY,29.51007,-98.580288,Medical Center,8,7,8,0,0.0,0,0,None,Normal
3,2021-05-16T00:10:26,63e0b0eae3495a39,ATT,AQM-01-00-01-00-01-00-GY,29.51007,-98.580288,Medical Center,8,7,8,0,0.0,0,0,None,Normal
4,2021-05-16T00:13:26,63e0b0eae3495a39,ATT,AQM-01-00-01-00-01-00-GY,29.51007,-98.580288,Medical Center,8,7,8,0,0.0,0,0,None,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23374,2021-05-01T23:35:00,70463,ITRON,ClimaCell,29.50484,-98.583500,Medical Center,-999,10,-999,-999,40.0,-999,11,NaN,Normal
23375,2021-05-01T23:40:00,70463,ITRON,ClimaCell,29.50484,-98.583500,Medical Center,-999,10,-999,-999,40.0,-999,11,NaN,Normal
23376,2021-05-01T23:45:00,70463,ITRON,ClimaCell,29.50484,-98.583500,Medical Center,-999,10,-999,-999,40.0,-999,11,NaN,Normal
23377,2021-05-01T23:50:00,70463,ITRON,ClimaCell,29.50484,-98.583500,Medical Center,-999,10,-999,-999,40.0,-999,11,NaN,Normal


In [8]:
air1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23379 entries, 0 to 23378
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DateTime        23379 non-null  object 
 1   Sensor_id       23379 non-null  object 
 2   Vendor          23379 non-null  object 
 3   SensorModel     23379 non-null  object 
 4   LAT             23379 non-null  float64
 5   LONG            23379 non-null  float64
 6   Zone            23379 non-null  object 
 7   Pm1_0           23379 non-null  int64  
 8   Pm2_5           23379 non-null  int64  
 9   Pm10            23379 non-null  int64  
 10  SO2             23379 non-null  int64  
 11  O3              23379 non-null  int64  
 12  CO              23379 non-null  int64  
 13  NO2             23379 non-null  int64  
 14  AlertTriggered  17619 non-null  object 
 15  SensorStatus    23379 non-null  object 
dtypes: float64(2), int64(7), object(7)
memory usage: 2.9+ MB


In [9]:
air1.O3 = air1.O3.astype('float64')

In [10]:
air1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23379 entries, 0 to 23378
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DateTime        23379 non-null  object 
 1   Sensor_id       23379 non-null  object 
 2   Vendor          23379 non-null  object 
 3   SensorModel     23379 non-null  object 
 4   LAT             23379 non-null  float64
 5   LONG            23379 non-null  float64
 6   Zone            23379 non-null  object 
 7   Pm1_0           23379 non-null  int64  
 8   Pm2_5           23379 non-null  int64  
 9   Pm10            23379 non-null  int64  
 10  SO2             23379 non-null  int64  
 11  O3              23379 non-null  float64
 12  CO              23379 non-null  int64  
 13  NO2             23379 non-null  int64  
 14  AlertTriggered  17619 non-null  object 
 15  SensorStatus    23379 non-null  object 
dtypes: float64(3), int64(6), object(7)
memory usage: 2.9+ MB
